In [13]:
import os
from dotenv import load_dotenv
import duckdb

# Load environment variables from the .env file
load_dotenv()

True

In [14]:
token = os.getenv("token")
workspace = os.getenv("workspace_url")
adls_conn = os.getenv("adls_conn")

In [3]:
conn = duckdb.connect()

In [12]:
conn.execute(f"""
INSTALL uc_catalog from core_nightly;
INSTALL delta from core;
LOAD delta;
LOAD uc_catalog;
CREATE SECRET (
	TYPE UC,
	TOKEN '${token}',
	ENDPOINT '${workspace}'
);
""")

In [15]:
conn.execute(f"""
INSTALL azure;
LOAD azure;
CREATE SECRET secret1 (
    TYPE AZURE,
    CONNECTION_STRING '${adls_conn}'
);
ATTACH 'dev' AS dev (TYPE UC_CATALOG);
""")

In [29]:

conn.sql("SELECT * FROM duckdb_databases();;")


┌───────────────┬──────────────┬─────────┬─────────┬───────────────────────┬──────────┬─────────┬──────────┐
│ database_name │ database_oid │  path   │ comment │         tags          │ internal │  type   │ readonly │
│    varchar    │    int64     │ varchar │ varchar │ map(varchar, varchar) │ boolean  │ varchar │ boolean  │
├───────────────┼──────────────┼─────────┼─────────┼───────────────────────┼──────────┼─────────┼──────────┤
│ dev           │         1838 │ dev     │ NULL    │ {}                    │ false    │ uc      │ false    │
│ memory        │         1146 │ NULL    │ NULL    │ {}                    │ false    │ duckdb  │ false    │
│ system        │            0 │ NULL    │ NULL    │ {}                    │ true     │ duckdb  │ false    │
│ temp          │         1828 │ NULL    │ NULL    │ {}                    │ true     │ duckdb  │ false    │
└───────────────┴──────────────┴─────────┴─────────┴───────────────────────┴──────────┴─────────┴──────────┘

In [30]:
conn.sql("select * from dev.silver.products;")

CatalogException: Catalog Error: Table with name products does not exist!
Did you mean "temp.pg_catalog.pg_proc"?